In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

# 1. Implement Multi-class SVM method (60 points)

In [5]:
def Multi_class_SVM(train_X,train_Y, test_x, max_ite=1000, LR=0.1, lamb=0.1, TOL=1e-5):
    
    loss_list= [np.inf]
    loss = 0
    ite = 0
    
    # Initialize weight and bias
    W = np.zeros((1,train_X.shape[1]))
    b = 0
    
    while ite <= max_ite or abs(loss-loss_list[-1])> TOL:
        
        margins = train_Y * (np.dot(train_X, W.T)+b)
        
        # Loss:
        loss = 0.5*lamb * np.sum(W**2) + (np.maximum(0, 1-margins).mean())
        
        # compute the gradient of the loss with respect to W and b
        grad_W = np.zeros_like(W)
        grad_b = 0
        for i in range(train_X.shape[0]):
            if margins[i]<1:
                grad_W += lamb * W - train_Y[i] * train_X[i]
                grad_b += -train_Y[i] 
            else:
                grad_W+=lamb * W
        grad_W /= train_X.shape[0]
        grad_b /= train_X.shape[0]
        
        # Update weight and bias w’ = w’ — ∝(∇J(w’))
        W -= LR*grad_W
        b -= LR*grad_b
        
        ite+=1
        loss_list.append(loss)
        
    return W, b, ite, loss_list
    

In [4]:
# From ChatGPT

import numpy as np

def svm_train(X_train, y_train, num_iterations, learning_rate, regularization_strength):
    # Initialize weight matrix and bias term
    W = np.zeros((1, X_train.shape[1]))
    b = 0

    # Train the model using gradient descent
    for iteration in range(num_iterations):
        # Compute the margin for each training example
        margins = y_train * (np.dot(X_train, W.T) + b)

        # Compute the hinge loss
        loss = np.maximum(0, 1 - margins)
        loss = np.mean(loss) + 0.5 * regularization_strength * np.sum(W**2)

        # Compute the gradient of the loss with respect to W and b
        grad_W = np.zeros_like(W)
        grad_b = 0
        for i in range(X_train.shape[0]):
            if margins[i] < 1:
                grad_W += regularization_strength * W - y_train[i] * X_train[i]
                grad_b -= y_train[i]
            else:
                grad_W += regularization_strength * W
        grad_W /= X_train.shape[0]
        grad_b /= X_train.shape[0]

        # Update the weight matrix and bias term using the gradient
        W -= learning_rate * grad_W
        b -= learning_rate * grad_b

    return W, b

def svm_predict(X_test, W, b):
    # Predict the labels of the test data
    scores = np.dot(X_test, W.T) + b
    y_pred = np.sign(scores).astype(int).ravel()

    return y_pred

# Assuming you have loaded your data into X and y arrays
X = np.load('X.npy')
y = np.load('y.npy')

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the SVM classifier on the training data
W, b = svm_train(X_train, y_train, num_iterations=1000, learning_rate=0.1, regularization_strength=0.1)

# Predict the labels of the test data
y_pred = svm_predict(X_test, W, b)

# Calculate the accuracy of the classifier on the test data
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)


FileNotFoundError: [Errno 2] No such file or directory: 'X.np'

* ref:https://towardsdatascience.com/svm-implementation-from-scratch-python-2db2fc52e5c2

# 2. Load train and test mat files, perform SVM and report acuracy on the test dataset (10 points)

In [6]:
trainmat = scipy.io.loadmat('train.mat')
testmat = scipy.io.loadmat('test.mat')
train_x, train_y = trainmat['features'], trainmat['labels']
test_x, test_y = testmat['features'], testmat['labels']
train_y = train_y.astype('float')
test_y = test_y.astype('float')

In [7]:
train_x.shape

(1123, 1000)

In [8]:
test_y

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
      

In [9]:
def predict(train_x, train_y, test_x, test_y):
    classes = np.unique(train_y)
    pred_list=[]
    for label in classes:
        one_hot_y=np.where(train_y==label,1.0,-1.0)
        W, b = Multi_class_SVM(train_x,one_hot_y, test_x)[:-2]
        score = np.dot(test_x, W.T) + b
        pred = np.sign(score).astype(int).ravel()
        pred_list.append(pred)
    pred_res = np.argmax(pred_list, axis=0)+1
    return pred_res

In [10]:
def accuracy(pred_res, train_y):
    count=0
    for i in range(len(test_y)):
        if pred_res[i]==test_y[i]:
            count+=1
    return round(count/len(test_y)*100,2)

In [11]:
pred_res = predict(train_x, train_y, test_x, test_y)

In [12]:
print('accuracy',accuracy(pred_res, train_y),'%')

accuracy 94.68 %


# 3. Reduce the dimensionality of features using PCA to low dimensions (e.g., 10, 3 ) and report the accuracy of the test datasets (10 points)

In [ ]:
for i in [3,10,20]:
    pca = PCA(n_components=i)
    reduced_train_x = pca.fit_transform(train_x)
    reduced_test_x = pca.fit_transform(test_x)
    pred_res = predict(reduced_train_x, train_y, reduced_test_x, test_y)
    
    print('dimensionality:',i)
    print('accuracy',accuracy(pred_res, train_y),'%')

dimensionality: 3
accuracy 18.48 %
dimensionality: 10
accuracy 39.04 %


# 4. Compare results with KNN, Decision Tree model (20 points)

In [ ]:
for i in range(1,10,2):    
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(train_x, train_y)
    pred = neigh.predict(test_x)
    print(f'n_neighbors = {i}')
    print('accuracy',accuracy(pred, train_y),'%')
    print()

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_x, train_y)
pred = clf.predict(test_x)
print('accuracy',accuracy(pred, train_y),'%')

__Decision Tree model has the lowest accuracy, while KNN has the highest accuracy that is similar with SVM model.__